# Carol Zhang

## Research Question/Interests

### Guest Experience & Satisfaction

Guest experience and satisfaction are crucial for Airbnb hosts since positive reviews can increase booking rates and listing visibility. Many travellers incorporate other guest reviews into their decision-making process when evaluating accommodation options. This research will explore key factors influencing guest satisfaction, such as pricing, host responsiveness, and amenities. By identifying patterns in high-rated listings, hosts can refine their offerings to improve guest experience and secure better reviews. 

### Key Research Questions
- Q1. How do listing features like the number of amenities or instant booking availability impact guest satisfaction ratings? 
- Q2. What is the relationship between the different ratings and the price of listings for different property types?
- Q3. Which amenities are most frequently associated with higher guest ratings in Airbnb listings?
- Q4. Do more descriptive listings have higher accuracy ratings? 

## EDA

#### Imports

In [28]:
import sys
import os

sys.path.append(os.path.abspath("../../code"))

import altair as alt
import pandas as pd
import ast
from toolz.curried import pipe

# Create a new data transformer that stores the files in a directory
# def json_dir(data, data_dir='altairdata'):
#     os.makedirs(data_dir, exist_ok=True)
#     return pipe(data, alt.to_json(filename=data_dir + '/{prefix}-{hash}.{extension}') )

# # Register and enable the new transformer
# alt.data_transformers.register('json_dir', json_dir)
# alt.data_transformers.enable('json_dir')

# Handle large data sets (default shows only 5000)
# See here: https://altair-viz.github.io/user_guide/data_transformers.html
alt.data_transformers.disable_max_rows()

alt.renderers.enable('jupyterlab')

from cleaning_workflows import prepare_dataset

#### Load in Data

In [2]:
df = pd.read_csv('../../data/raw/listings.csv', parse_dates=['first_review', 'last_review'])
# df = prepare_dataset(df)
df.head()

,id,listing_url,scrape_id,last_scraped,source,name,description,neighborhood_overview,picture_url,host_id,...,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,39572,https://www.airbnb.com/rooms/39572,20250103033441,2025-01-03,city scrape,1 br in a 2 br apt (Midtown West),NaN,NaN,https://a0.muscache.com/pictures/fd1bffd9-ccf8...,169927,...,5.00,4.98,4.86,NaN,f,2,1,1,0,0.25
1,39593,https://www.airbnb.com/rooms/39593,20250103033441,2025-01-03,city scrape,A lovely room w/ a Manhattan view,"A private, furnished large room to rent Jan/F...","Nate Silver called this super safe, clean, qui...",https://a0.muscache.com/pictures/0b9110f7-3b24...,110506,...,4.96,4.79,4.93,NaN,f,1,0,1,0,0.20
2,39704,https://www.airbnb.com/rooms/39704,20250103033441,2025-01-03,previous scrape,"Private, Large & Sunny 1BR w/W&D",It's a No Brainer:<br />•Terrific Space For Le...,The Neighborhood<br />• Rich History <br />• B...,https://a0.muscache.com/pictures/0bc4e8a4-c047...,170510,...,4.92,4.38,4.72,NaN,f,2,2,0,0,1.93
3,42300,https://www.airbnb.com/rooms/42300,20250103033441,2025-01-03,city scrape,Beautiful Lower East Side Loft,Architect-owned loft is a corner unit in a bea...,"The apartment is in the border of Soho, LES an...",https://a0.muscache.com/pictures/0e285e13-ee14...,184755,...,4.87,4.57,4.62,NaN,f,1,1,0,0,0.40
4,42729,https://www.airbnb.com/rooms/42729,20250103033441,2025-01-03,city scrape,@HouseOnHenrySt - Private 2nd bedroom w/shared...,NaN,"Lovely old Brooklyn neighborhood, with brick/b...",https://a0.muscache.com/pictures/925fe213-f5e1...,11481,...,4.73,4.58,4.64,NaN,f,4,1,3,0,1.26


#### Data Cleaning

In [3]:
# Make a copy of the dataset
df_cleaned = df.copy()

In [4]:
# Drop unnecessary columns 
drop_columns = [
    'id',  # Unique identifier 
    'scrape_id',  # Only one unique value, provides no variation
    'last_scraped',  # Only one unique value, not useful for analysis
    'source',  # Only two unique values, not relevant for analysis
    'neighbourhood',  # No clear semantic meaning
    'calendar_updated',  # No clear semantic meaning, all values missing
    'license',  # Mostly missing values, not useful for analysis
    'host_name', # Not useful for analysis 
    'host_thumbnail_url',  # Just a link, not useful for analysis
    'host_picture_url',  # Just a link, not useful for analysis
    'host_url',  # Just a link, not useful for analysis
    'listing_url',  # Just a link, not useful for analysis
    'picture_url',  # Just a link, not useful for analysis
    'bathrooms_text',  # Redundant, `bathrooms` already provides numerical info
    'minimum_minimum_nights',  # Redundant, covered by `minimum_nights`
    'maximum_minimum_nights',  # Redundant, covered by `minimum_nights`
    'minimum_maximum_nights',  # Redundant, covered by `maximum_nights`
    'maximum_maximum_nights',  # Redundant, covered by `maximum_nights`
    'calendar_last_scraped'  # Similar to `last_scraped`, unnecessary
]

df_cleaned.drop(columns=drop_columns, inplace=True)

In [5]:
# Fill missing values for categorical variables
categorical_fill_values = {
    "name": "Unknown",
    "description": "No description available",
    "neighborhood_overview": "No overview available",
    "host_name": "Unknown",
    "host_location": "Not provided",
    "host_response_time": "No response time",
    "host_is_superhost": "f",
    "has_availability": "f",
    "instant_bookable": "f"
}

df_cleaned.fillna(value=categorical_fill_values, inplace=True)

In [6]:
# Convert date columns
date_cols = ["host_since", "first_review", "last_review"]
for col in date_cols:
    df_cleaned[col] = pd.to_datetime(df_cleaned[col], errors="coerce")
    df_cleaned[col] = df_cleaned[col].fillna(df_cleaned[col].mode()[0])

In [7]:
# Convert percentage columns to numeric
percentage_cols = ["host_response_rate", "host_acceptance_rate"]

for col in percentage_cols:
    df_cleaned[col] = df_cleaned[col].astype(str).str.replace("%", "").astype(float) / 100
    # Fill missing values with the median
    df_cleaned[col] = df_cleaned[col].fillna(df_cleaned[col].median())

In [8]:
# Convert the price column to string type, remove "$" and "," and convert to float
df_cleaned["price"] = df_cleaned["price"].astype(str)
df_cleaned["price"] = df_cleaned["price"].str.replace(r"[\$,]", "", regex=True).astype(float)

# Fill missing values with the median 
df_cleaned["price"] = df_cleaned["price"].fillna(df_cleaned["price"].median())

In [9]:
# Convert 't' and 'f' values to boolean (True/False) for binary columns
binary_columns = ['instant_bookable', 'host_is_superhost', 'host_has_profile_pic', 'host_identity_verified', 'has_availability']

for col in binary_columns:
    df_cleaned[col] = df_cleaned[col].map({'t': True, 'f': False})

In [10]:
# Fill missing numerical values with median
numerical_cols = df_cleaned.select_dtypes(include=["number"]).columns.tolist()
for col in numerical_cols:
    df_cleaned[col] = df_cleaned[col].fillna(df_cleaned[col].median())

# Fill missing values in categorical columns with the most common value
categorical_cols = df_cleaned.select_dtypes(include=['object']).columns
df_cleaned[categorical_cols] = df_cleaned[categorical_cols].fillna(df_cleaned[categorical_cols].mode().iloc[0])

# Strip leading/trailing spaces from string columns
df_cleaned[categorical_cols] = df_cleaned[categorical_cols].apply(lambda col: col.astype(str).str.strip())

C:\Users\Carol\AppData\Local\Temp\ipykernel_9184\3760752339.py:8: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_cleaned[categorical_cols] = df_cleaned[categorical_cols].fillna(df_cleaned[categorical_cols].mode().iloc[0])


In [11]:
df_cleaned.head()

,name,description,neighborhood_overview,host_id,host_since,host_location,host_about,host_response_time,host_response_rate,host_acceptance_rate,...,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,1 br in a 2 br apt (Midtown West),No description available,No overview available,169927,2010-07-17,"Saint-Aubin-sur-Scie, France","Facebook Likes:\r\nNew York French Geek, David...",within a day,1.00,0.88,...,4.98,5.00,4.98,4.86,False,2,1,1,0,0.25
1,A lovely room w/ a Manhattan view,"A private, furnished large room to rent Jan/F...","Nate Silver called this super safe, clean, qui...",110506,2010-04-18,"New York, NY","I grew up in South Korea, moved to Montreal, C...",within a few hours,1.00,0.60,...,4.96,4.96,4.79,4.93,False,1,0,1,0,0.20
2,"Private, Large & Sunny 1BR w/W&D",It's a No Brainer:<br />•Terrific Space For Le...,The Neighborhood<br />• Rich History <br />• B...,170510,2010-07-18,"New York, United States",I am a self employed licensed real estate brok...,No response time,1.00,0.88,...,4.89,4.92,4.38,4.72,False,2,2,0,0,1.93
3,Beautiful Lower East Side Loft,Architect-owned loft is a corner unit in a bea...,"The apartment is in the border of Soho, LES an...",184755,2010-07-29,"New York, NY",I am an architect living in NYC and have my ow...,within a day,1.00,1.00,...,4.85,4.87,4.57,4.62,False,1,1,0,0,0.40
4,@HouseOnHenrySt - Private 2nd bedroom w/shared...,No description available,"Lovely old Brooklyn neighborhood, with brick/b...",11481,2009-03-26,"New York, NY",I have been a host with Airbnb since its intro...,within a day,0.67,0.33,...,4.71,4.73,4.58,4.64,False,4,1,3,0,1.26


#### Summary Statistics 

In [12]:
df_cleaned.shape

(37784, 56)

In [13]:
df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37784 entries, 0 to 37783
Data columns (total 56 columns):
 #   Column                                        Non-Null Count  Dtype         
---  ------                                        --------------  -----         
 0   name                                          37784 non-null  object        
 1   description                                   37784 non-null  object        
 2   neighborhood_overview                         37784 non-null  object        
 3   host_id                                       37784 non-null  int64         
 4   host_since                                    37784 non-null  datetime64[ns]
 5   host_location                                 37784 non-null  object        
 6   host_about                                    37784 non-null  object        
 7   host_response_time                            37784 non-null  object        
 8   host_response_rate                            37784 non-null  floa

In [14]:
df_cleaned.isnull().sum()

name                                            0
description                                     0
neighborhood_overview                           0
host_id                                         0
host_since                                      0
host_location                                   0
host_about                                      0
host_response_time                              0
host_response_rate                              0
host_acceptance_rate                            0
host_is_superhost                               0
host_neighbourhood                              0
host_listings_count                             0
host_total_listings_count                       0
host_verifications                              0
host_has_profile_pic                            0
host_identity_verified                          0
neighbourhood_cleansed                          0
neighbourhood_group_cleansed                    0
latitude                                        0


**Comments** 
- There are 377,784 observations and 56 columns after cleaning 
- There are no missing values in the dataset after imputation 

#### Feature Engineering

**`amenities`**

Currently, amenities are stored as a list of strings. To make this variable more useful, I want to apply the following transformations: 
1. **Convert the list of amenities into separate binary columns**, where each column represents whether a specific amenity is included in a listing (1 for yes, 0 for no). This will help answer Q3 by identifying which amenities are the most common and their correlation with higher-rated listings. 
2. **Create a new column counting the number of amenities** for each listing based on the number of elements in the `amenities` list. This numerical value will be used to answer Q1 to help us explore whether listings with more amenities receive higher ratings. 

In [15]:
# Display the amenities column
print(df_cleaned['amenities'])

0        ["Hangers", "Wifi", "Smoke alarm", "Carbon mon...
1        ["Hangers", "Smoke alarm", "Hair dryer", "Clot...
2        ["Hangers", "Coffee maker", "Smoke alarm", "Fi...
3        ["Hangers", "Coffee maker", "Smoke alarm", "Ha...
4        ["Hangers", "Coffee maker", "Smoke alarm", "TV...
                               ...                        
37779    ["Hangers", "Smoke alarm", "Fire extinguisher"...
37780    ["Dedicated workspace", "HDTV with Amazon Prim...
37781    ["Hangers", "Smoke alarm", "Stove", "Cooking b...
37782    ["Wifi", "Central air conditioning", "Hot wate...
37783    ["Wifi", "TV", "Dedicated workspace", "Carbon ...
Name: amenities, Length: 37784, dtype: object


In [16]:
# Transformation 1: Convert the list of amenities into separate binary columns
# Convert the string representation of lists back into actual lists
df_cleaned['amenities'] = df_cleaned['amenities'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

# Calculate the frequency of each amenity
amenity_counts = df_cleaned['amenities'].explode().value_counts()

# Select the top 25 amenities by frequency
top_25_amenities = amenity_counts.head(25).index.tolist()

# Get the top 25 amenities and their count
top_25_amenities_table = amenity_counts.head(25).reset_index()
top_25_amenities_table.columns = ['Amenity', 'Count']

top_25_amenities_table

,Amenity,Count
0,Wifi,35169
1,Smoke alarm,34232
2,Kitchen,32889
3,Carbon monoxide alarm,29881
4,Essentials,28900
5,Heating,26301
6,Hangers,26254
7,Air conditioning,26158
8,Hot water,25006
9,Hair dryer,23183


In [17]:
# Concatenate the new amenity columns to df_cleaned
amenity_columns = pd.DataFrame(
    [[1 if amenity in x else 0 for amenity in top_25_amenities] for x in df_cleaned['amenities']],
    columns=top_25_amenities)

df_cleaned = pd.concat([df_cleaned, amenity_columns], axis=1)

In [18]:
# Transformation 2: Create a new column counting the number of amenities in each listing 
df_cleaned['num_amenities'] = df_cleaned['amenities'].apply(len)
print(df_cleaned[['amenities', 'num_amenities']].head())

                                           amenities  num_amenities
0  [Hangers, Wifi, Smoke alarm, Carbon monoxide a...             14
1  [Hangers, Smoke alarm, Hair dryer, Clothing st...             34
2  [Hangers, Coffee maker, Smoke alarm, Fire exti...             29
3  [Hangers, Coffee maker, Smoke alarm, Hair drye...             27
4  [Hangers, Coffee maker, Smoke alarm, TV with s...             23


**`name`, `description`, `neighborhood_overview`**

In Q4, I want to explore whether more descriptive listings have higher accuracy ratings (`review_scores_accuracy`). To quantify the descriptiveness of a listing, I will **create new columns to count the number of words in the string values** for the `name`, `description`, and `neighborhood_overview` variables. This will serve as a proxy for the number of details that are included in the listing. For example, a property with the name "2 Bedroom Apartment" (word count = 3) is less descriptive than "Sunny, beautiful 2 Bedroom Apartment in Downtown Manhatten" (word count = 8). 

In [19]:
# Function to count the number of words in a string
def word_count(text):
    if isinstance(text, str):
        return len(text.split())
    return 0  # If the text is not a string, return 0

df_cleaned['name_word_count'] = df_cleaned['name'].apply(word_count)
df_cleaned['description_word_count'] = df_cleaned['description'].apply(word_count)
df_cleaned['neighborhood_overview_word_count'] = df_cleaned['neighborhood_overview'].apply(word_count)

print(df_cleaned[['name_word_count', 'description_word_count', 'neighborhood_overview_word_count']].head())

   name_word_count  description_word_count  neighborhood_overview_word_count
0                9                       3                                 3
1                7                      58                               153
2                6                      32                                68
3                5                      88                                21
4                7                       3                                16


#### Univariate Analysis

In [20]:
# Describe quantitative columns, excluding binary columns for amenities and other unmeaningful columns
exclude_columns = top_25_amenities + ['latitude', 'longitude', 'host_id']
df_numerical = df_cleaned.drop(columns=exclude_columns)

pd.set_option('display.max_columns', None)
df_numerical.describe()

,host_since,host_response_rate,host_acceptance_rate,host_listings_count,host_total_listings_count,accommodates,bathrooms,bedrooms,beds,price,minimum_nights,maximum_nights,minimum_nights_avg_ntm,maximum_nights_avg_ntm,availability_30,availability_60,availability_90,availability_365,number_of_reviews,number_of_reviews_ltm,number_of_reviews_l30d,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month,num_amenities,name_word_count,description_word_count,neighborhood_overview_word_count
count,37784,37784.000000,37784.000000,37784.000000,37784.000000,37784.000000,37784.000000,37784.000000,37784.000000,37784.000000,37784.000000,3.778400e+04,37784.000000,3.778400e+04,37784.000000,37784.000000,37784.000000,37784.000000,37784.000000,37784.000000,37784.000000,37784,37784,37784.000000,37784.000000,37784.000000,37784.000000,37784.000000,37784.000000,37784.000000,37784.000000,37784.000000,37784.000000,37784.000000,37784.000000,37784.000000,37784.000000,37784.000000,37784.000000
mean,2017-03-06 09:48:26.373067776,0.952508,0.812859,263.054864,351.429229,2.754896,1.119654,1.321723,1.383946,170.434126,28.882172,5.793950e+04,29.497840,8.501277e+05,12.314075,27.011275,42.416552,163.400963,25.658639,3.731447,0.294357,2021-02-19 02:42:05.259369216,2023-05-11 06:14:56.379419904,4.763849,4.805924,4.704505,4.870094,4.867555,4.775639,4.676217,71.636354,45.435555,23.985232,0.005187,0.685365,25.991663,6.148052,57.270035,36.569289
min,2008-08-11 00:00:00,0.000000,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,8.000000,1.000000,1.000000e+00,1.000000,1.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2009-05-25 00:00:00,2011-05-12 00:00:00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.010000,0.000000,1.000000,1.000000,1.000000
25%,2014-07-07 00:00:00,1.000000,0.840000,1.000000,1.000000,2.000000,1.000000,1.000000,1.000000,113.750000,30.000000,1.500000e+02,30.000000,3.650000e+02,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2019-01-21 00:00:00,2023-03-19 00:00:00,4.750000,4.817500,4.690000,4.890000,4.900000,4.750000,4.670000,1.000000,0.000000,0.000000,0.000000,0.160000,13.000000,5.000000,39.000000,3.000000
50%,2016-08-02 00:00:00,1.000000,0.880000,2.000000,3.000000,2.000000,1.000000,1.000000,1.000000,132.000000,30.000000,3.650000e+02,30.000000,5.676500e+02,2.000000,23.000000,41.000000,155.000000,3.000000,0.000000,0.000000,2022-12-11 00:00:00,2024-10-11 00:00:00,4.850000,4.900000,4.810000,4.950000,4.960000,4.850000,4.760000,2.000000,1.000000,1.000000,0.000000,0.290000,24.000000,6.000000,62.000000,10.000000
75%,2019-10-29 00:00:00,1.000000,0.950000,10.000000,15.000000,4.000000,1.000000,1.000000,1.000000,156.000000,30.000000,1.125000e+03,30.000000,1.125000e+03,29.000000,58.000000,88.000000,329.000000,22.000000,1.000000,0.000000,2023-01-01 00:00:00,2024-12-15 00:00:00,4.940000,4.970000,4.910000,5.000000,5.000000,4.950000,4.850000,9.000000,2.000000,2.000000,0.000000,0.550000,36.000000,8.000000,79.000000,60.000000
max,2024-12-27 00:00:00,1.000000,1.000000,5079.000000,9048.000000,16.000000,15.500000,16.000000,42.000000,20000.000000,1250.000000,2.147484e+09,1250.000000,2.147484e+09,30.000000,60.000000,90.000000,365.000000,2485.000000,1779.000000,137.000000,2025-01-02 00:00:00,2025-01-02 00:00:00,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,1154.000000,1154.000000,739.000000,4.000000,116.300000,95.000000,47.000000,185.000000,227.000000
std,NaN,0.173547,0.232501,1000.361076,1196.815995,1.903900,0.442243,0.869987,0.984719,277.145183,29.905150,1.104828e+07,30.858242,4.252198e+07,13.408262,26.656382,39.790892,148.521232,62.619846,18.79

In [21]:
# Nominal Features
df_cleaned.describe(include=['object'])

,name,description,neighborhood_overview,host_location,host_about,host_response_time,host_neighbourhood,host_verifications,host_has_profile_pic,host_identity_verified,neighbourhood_cleansed,neighbourhood_group_cleansed,property_type,room_type,amenities
count,37784,37784,37784,37784,37784,37784,37784,37784,37784,37784,37784,37784,37784,37784,37784
unique,36058,31140,15117,988,11654,5,521,7,2,2,223,5,69,4,30453
top,Water View King Bed Hotel Room,No description available,No overview available,"New York, NY","We’re Blueground, a global proptech company wi...",No response time,Bedford-Stuyvesant,"['email', 'phone']",True,True,Bedford-Stuyvesant,Manhattan,Entire rental unit,Entire home/apt,"[Wifi, TV, Smoke alarm, Carbon monoxide alarm,..."
freq,30,973,17177,22469,17326,15585,9498,29303,36695,33339,2678,16819,15887,20160,237


**Comments** 
- Review scores are generally high, with an average score of around 4.8 for each category
- The average number of amenities listed per property is 25.99
- Availability is relatively high for most properties, with an average availability of 163 days in a year
- The mean price is $170.43, with a large range from 8 to 20,000 which indicates that there are both budget and luxury properties listed
- The average number of reviews is 25.66, with outliers having as many as 2485 reviews

In [44]:
chart = alt.Chart(df_cleaned).mark_bar().encode(
    x='room_type:N',
    y='count():Q'
)
chart

UnicodeEncodeError: 'utf-8' codec can't encode characters in position 27-28: surrogates not allowed

alt.Chart(...)

In [38]:
# Histogram for distribution of guest ratings 
ratings_hist = alt.Chart(df_cleaned).mark_bar().encode(
    x='review_scores_rating:Q',  
    y='count():Q',  
).properties(
    title='Distribution of Guest Satisfaction Ratings')

ratings_hist

UnicodeEncodeError: 'utf-8' codec can't encode characters in position 27-28: surrogates not allowed

alt.Chart(...)

#### Multivariate Analysis

In [23]:
# Bar chart of Accident Cause vs Economic Loss 
causes_barchart = alt.Chart(accidents).mark_bar().encode(
    alt.Y('Accident Cause:N', sort='-x'),
    alt.X('average(Economic Loss):Q', title="Average Economic Loss ($)", scale=alt.Scale(domain=[49000, 51000])),
    alt.Color('Accident Cause:N')
).properties(
    title="Average Economic Loss by Accident Cause", 
    height=300,
    width=400)

causes_zoomed = causes_barchart.encode(alt.X('average(Economic Loss):Q', title="Average Economic Loss ($)")
                                      ).properties(title="Average Economic Loss by Accident Cause [Zoomed In]")

causes_barchart  | causes_zoomed

NameError: name 'accidents' is not defined

From the bar charts above, it seems like Economic Loss doesn't vary much overall by Accident Cause and it isn't a good predictor for the cost of an accident. The difference between the cause with the highest average economic loss (Drunk Driving) and lowest (Speeding) is only $619.35.

In [ ]:
# Boxplot of how Medical Costs vary by Accident Severity 
boxplot_medical_costs = alt.Chart(accidents).mark_boxplot().encode(
    alt.X('Accident Severity:N', title='Accident Severity'),  
    alt.Y('Medical Cost:Q', title='Medical Cost ($)'),       
    alt.Color('Accident Severity:N', legend=None),       
    tooltip=['Accident Severity', 'Medical Cost']               
).properties(
    title="Medical Costs by Accident Severity", 
    height=400,  
    width=500)

boxplot_medical_costs

It doesn't look like Medical Costs vary by Accident Severity. All three levels of severity have near-identical distributions, max and min values, whiskers, and median values. 

In [ ]:
# Scatterplot of Medical Costs and Economic Loss
scatter_medical_economic = alt.Chart(accidents).mark_point().encode(
    alt.X('Medical Cost:Q'), 
    alt.Y('Economic Loss:Q'),     
    alt.Color('Accident Severity:N', legend=None),
    tooltip=['Medical Cost', 'Economic Loss', 'Accident Severity']
).properties(
    title="Medical Cost vs Economic Loss by Accident Severity",
    height=400,  
    width=500) 

scatter_medical_economic

There are no relationships in the scatterplot above. 

In [ ]:
# Heat map for columns related to Economic Loss and Medical Costs 

select_columns = ['Economic Loss', 'Medical Cost', 'Number of Vehicles Involved', 'Number of Fatalities', 'Insurance Claims']

corr_matrix = accidents[select_columns].corr()
corr_matrix_melted = corr_matrix.reset_index().melt(id_vars='index')
corr_matrix_melted.columns = ['Variable1', 'Variable2', 'Correlation']

heatmap = alt.Chart(corr_matrix_melted).mark_rect().encode(
    alt.X('Variable1:N', title='Variable 1'),
    alt.Y('Variable2:N', title='Variable 2'),
    alt.Color('Correlation:Q', scale=alt.Scale(scheme='viridis'), title='Correlation'),
    tooltip=['Variable1', 'Variable2', 'Correlation']
).properties(
    title="Correlation Heatmap",
    height=400,
    width=400)

heatmap

There are no correlations between the variables. 

### Task Analysis

#### Which accident-related factors contribute the most to economic loss, and how does their impact vary by accident severity?
- **Retrieve Value**: Extract Economic Loss, Accident Severity, and potential factors like Number of Vehicles Involved and Accident Cause.
- **Sort**: Sort Economic Loss from high to low to find highest contributing factors. 
- **Filter**: Filter accidents based on Accident Severity.
- **Compute Derived Value**: Calculate the average Economic Loss for categorical variables.
- **Correlate**: Analyze the relationship between Economic Loss and other variables.
- **Characterize Distribution**: Visualize how Economic Loss is distributed across different Accident Severity levels and other factors.
- **Determine Range**: Identify the range of Economic Loss for each Accident Severity level and assess how the range changes across different contributing factors.

#### How do medical costs and total economic loss per accident differ between urban and rural areas?
- **Retrieve Value**: Extract Medical Costs, Economic Loss, and Urban/Rural classifications.
- **Filter**: Filter accidents into Urban and Rural areas.
- **Correlate**: Analyze the relationship between Urban/Rural classification and Medical Costs or Economic Loss.
- **Cluster**: Group accidents based on Urban/Rural and other factors (like Accident Severity) to see if different areas show different patterns of cost.

#### Do speed limits impact the medical costs and economic loss associated with road accidents?
- **Retrieve Value**: Extract Speed Limits, Medical Costs, and Economic Loss for each accident.
- **Characterize Distribution**: Visualize the distribution of Economic Loss and Medical Costs for different Speed Limits.
- **Correlate**: Analyze the relationship between Speed Limit and Medical Costs or Economic Loss, identifying if higher speed limits are correlated with higher costs.